In [1]:
import csv
from tabulate import tabulate

# configure below to print tables for Jupyter notebook or for Latex
tablestyle = "fancy_grid"
#tablestyle = "latex"

def nano_to_milli(x):
    return round(x * 10**(-6),1)

error_margin = 0.03

In [2]:
header = ['alg','opt','dim','len','max','neg','time','unit','dev']
table_vals = []

with open('strat01a/data.csv', newline='') as csvfile:
    results_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    ctr = 0
    for row in results_reader:
        #print(', '.join(row))
        if ctr > 0:
            table_vals.append(row)
        ctr += 1
        
#print(tabulate(row, headers=header, tablefmt=tablestyle))

In [3]:
def collect_pos_vals(header_val):
    lis_vals = []
    for row in table_vals:
        tmp = row[header.index(header_val)]
        if not tmp in lis_vals:
            lis_vals.append(tmp)
    return lis_vals
        
algs = collect_pos_vals('alg')
opts = collect_pos_vals('opt')
dims = collect_pos_vals('dim')
lens = collect_pos_vals('len')
size_or_length = collect_pos_vals('max')
neg_degrees = collect_pos_vals('neg')

In [4]:
def collect_corresponding_val(opt_val, algo, dupl_dim, fixed_size, neg_degree, nr_diff):
    for row in table_vals:
        test0 = row[header.index('opt')] == opt_val
        test1 = row[header.index('alg')] == algo
        test2 = row[header.index('dim')] == dupl_dim
        test3 = row[header.index('max')] == fixed_size
        test4 = row[header.index('neg')] == neg_degree
        test5 = row[header.index('len')] == nr_diff
        if test0 and test1 and test2 and test3 and test4 and test5:
            return int(row[header.index('time')])
    print('not found')
    return 10**10

In [5]:
def compare_opts(algo, neg_degree):
    #variable = lens
    #variable = [lens[i-1] for i in [1,10,20,30,40,50,60,70,80,90,100]]
    variable = [lens[i-1] for i in [1,10,20,30,60,100]]
    caption = "Comparison of different opts for " + algo + " and various policy lengths and negation degree " + str(neg_degree)
    print(caption)
    
    target_table = []
    max_factor = []
    for opt_val in opts:
        target_table.append([opt_val])
        max_factor.append(0)
    
    for len_val in variable:
        min_opt = opts[0]
        min_val = 10**11
        for opt_val in opts:
            val = collect_corresponding_val(opt_val, algo, dims[0], size_or_length[0], neg_degree, len_val)
            if val < min_val:
                min_val = val
                min_opt = opt_val
            target_table[opts.index(opt_val)].append(val)
        for opt_val in opts:
            if opt_val == min_opt:
                target_table[opts.index(opt_val)].append('-')
            else:
                test_val = target_table[opts.index(opt_val)][-1]
                sf = round(test_val/min_val,3)
                target_table[opts.index(opt_val)].append(sf)
                #target_table[opts.index(opt_val)].append(round((test_val - min_val)/min_val * 100,1))
                if sf > max_factor[opts.index(opt_val)]:
                    max_factor[opts.index(opt_val)] = sf
            target_table[opts.index(opt_val)][-2] = nano_to_milli(target_table[opts.index(opt_val)][-2])
    
    header = ['Opt']
    for var in variable:
        header.append(var)
        header.append('SF')
                
    print(tabulate(target_table, headers=header, tablefmt=tablestyle))
    
    print("Max factors")
    for opt_val in opts:
        print(opt_val, max_factor[opts.index(opt_val)])

In [6]:
def analyze_all():
    for n_d in neg_degrees:
        for algo in algs[1:]:
            compare_opts(algo, n_d)
            for opt_val in opts:
                val = collect_corresponding_val(opt_val, algo, dims[0], size_or_length[0], n_d, lens[-1])
                val0 = collect_corresponding_val(opt_val, algo, dims[0], size_or_length[0], n_d, lens[0])
                average = (val-val0)/99
                print("Average growth for " + opt_val, nano_to_milli(average))
                cons = nano_to_milli(val0 - average)
                if cons > 0.01:
                    print("Constant costs are ", cons)
                else:
                    print("Constant costs are close to 0")
#analyze_all()

In [7]:
def print_all_costs_at_60():
    variable = algs[1:]
    
    target_table = []
    for algo in variable:
        target_table.append([algo + '(pos)'])
        target_table.append([algo + '(neg)'])
    
    for algo in variable:
        for nd in neg_degrees[0:2]:
            for opt_val in opts:
                val = collect_corresponding_val(opt_val, algo, dims[0], size_or_length[0], nd, lens[-1])
                if nd == neg_degrees[0]:
                    ad = 0
                else:
                    ad = 1
                target_table[2*variable.index(algo) + ad].append(nano_to_milli(val))
    
    header = ['alg']
    for opt_val in opts:
        header.append(opt_val)
                
    print(tabulate(target_table, headers=header, tablefmt=tablestyle))

In [8]:
print_all_costs_at_60()

╒══════════════╤════════╤════════╤════════╤════════╤════════╤════════╤════════╕
│ alg          │   opt0 │   opt1 │   opt2 │   opt3 │   opt4 │   opt5 │   opt6 │
╞══════════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╡
│ keygen(pos)  │  420.7 │  420.9 │  420.9 │  421.1 │  444.8 │  561.7 │  585.4 │
├──────────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│ keygen(neg)  │  421.5 │  420.8 │  421   │  421   │  444.7 │  561.2 │  585.2 │
├──────────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│ encrypt(pos) │  465.5 │  464.6 │  465.3 │  390.1 │  413.7 │  389.1 │  414.8 │
├──────────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│ encrypt(neg) │  465.7 │  464.3 │  465.4 │  390.9 │  391.5 │  442.3 │  446.4 │
├──────────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│ decrypt(pos) │  320.9 │  320.2 │  320.5 │  163.8 │  163.9 │  164   │  164.2 │
├──────────────┼────────┼────────┼──────

In [9]:
def print_all_costs_at_60_flip_table():
    variable = opts
    
    target_table = []
    for opt_val in variable:
        target_table.append([opt_val + '(pos)'])
        target_table.append([opt_val + '(neg)'])
    
    for algo in algs[1:]:
        for nd in neg_degrees[0:2]:
            min_opt = opts[0]
            min_val = 10**11
            if nd == neg_degrees[0]:
                ad = 0
            else:
                ad = 1
                
            for opt_val in opts:
                val = collect_corresponding_val(opt_val, algo, dims[0], size_or_length[0], nd, lens[-1])
                if val < min_val:
                    min_val = val
                    min_opt = opt_val
                target_table[2*variable.index(opt_val) + ad].append(val)
            
            for opt_val in opts:
                if opt_val == min_opt:
                    target_table[2*variable.index(opt_val) + ad].append('-')
                else:
                    test_val = target_table[2*variable.index(opt_val) + ad][-1]
                    sf = round(test_val/min_val,3)
                    target_table[2*variable.index(opt_val) + ad].append(sf)
                target_table[2*variable.index(opt_val) + ad][-2] = nano_to_milli(target_table[2*variable.index(opt_val) + ad][-2])
            
    
    header = ['opt']
    header2 = ['']
    for algo in algs[1:]:
        header.append(algo)
        header2.append('cost')
        header.append(algo)
        header2.append('SF')
                
    print(tabulate([header2] + target_table, headers=header, tablefmt=tablestyle))

In [10]:
print_all_costs_at_60_flip_table()

╒═══════════╤══════════╤══════════╤═══════════╤═══════════╤═══════════╤═══════════╕
│ opt       │ keygen   │ keygen   │ encrypt   │ encrypt   │ decrypt   │ decrypt   │
╞═══════════╪══════════╪══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│           │ cost     │ SF       │ cost      │ SF        │ cost      │ SF        │
├───────────┼──────────┼──────────┼───────────┼───────────┼───────────┼───────────┤
│ opt0(pos) │ 420.7    │ -        │ 465.5     │ 1.196     │ 320.9     │ 1.959     │
├───────────┼──────────┼──────────┼───────────┼───────────┼───────────┼───────────┤
│ opt0(neg) │ 421.5    │ 1.002    │ 465.7     │ 1.191     │ 610.9     │ 2.063     │
├───────────┼──────────┼──────────┼───────────┼───────────┼───────────┼───────────┤
│ opt1(pos) │ 420.9    │ 1.0      │ 464.6     │ 1.194     │ 320.2     │ 1.954     │
├───────────┼──────────┼──────────┼───────────┼───────────┼───────────┼───────────┤
│ opt1(neg) │ 420.8    │ -        │ 464.3     │ 1.188     │ 452.3     │ 1.52